In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

In [2]:
app=Dash(__name__)
sd=pd.read_excel("C:\\Users\\kiran\\OneDrive\\Desktop\\EDA Worksheet.xlsx")
sd.head()

,OrderNumber,Sales Channel,OrderDate,order_year,DeliveryDate,_SalesTeamID,Sales Rep,_CustomerID,customer_name,_StoreID,state,state_code,store_region,store_city,_ProductID,product_name,Order Quantity,Unit Price,Unit Cost,total_sales
0,SO - 0007632,In-Store,2020-11-06,2020,2020-11-21,4,Chris Armstrong,41,Weimei Corp,3,Alabama,AL,South,Mobile,25,TV and video,3,3939.6,1851.612,11818.8
1,SO - 0007587,In-Store,2020-11-01,2020,2020-11-12,2,Keith Griffin,5,Procter Corp,1,Alabama,AL,South,Birmingham,3,Table Lamps,2,2391.9,1363.383,4783.8
2,SO - 0007230,In-Store,2020-09-21,2020,2020-10-12,3,Jerry Green,50,Sundial,4,Alabama,AL,South,Montgomery,8,Cocktail Glasses,5,3899.4,3275.496,19497.0
3,SO - 0007187,Wholesale,2020-09-17,2020,2020-09-26,26,Donald Reynolds,12,Medline,3,Alabama,AL,South,Mobile,7,Dinnerware,8,1031.8,588.126,8254.4
4,SO - 0007183,Distributor,2020-09-17,2020,2020-10-02,24,Roy Rice,13,Ole Group,3,Alabama,AL,South,Mobile,34,Bedroom Furniture,3,268.0,222.440,804.0


In [3]:
sd_copy=sd.copy()
sd_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7991 entries, 0 to 7990
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   OrderNumber     7991 non-null   object        
 1   Sales Channel   7991 non-null   object        
 2   OrderDate       7991 non-null   datetime64[ns]
 3   order_year      7991 non-null   int64         
 4   DeliveryDate    7991 non-null   datetime64[ns]
 5   _SalesTeamID    7991 non-null   int64         
 6   Sales Rep       7991 non-null   object        
 7   _CustomerID     7991 non-null   int64         
 8   customer_name   7991 non-null   object        
 9   _StoreID        7991 non-null   int64         
 10  state           7991 non-null   object        
 11  state_code      7991 non-null   object        
 12  store_region    7991 non-null   object        
 13  store_city      7991 non-null   object        
 14  _ProductID      7991 non-null   int64         
 15  prod

In [4]:
sd_copy.columns

Index(['OrderNumber', 'Sales Channel', 'OrderDate', 'order_year',
       'DeliveryDate', '_SalesTeamID', 'Sales Rep', '_CustomerID',
       'customer_name', '_StoreID', 'state', 'state_code', 'store_region',
       'store_city', '_ProductID', 'product_name', 'Order Quantity',
       'Unit Price', 'Unit Cost', 'total_sales'],
      dtype='object')

In [5]:
sd_totalstate=(sd_copy.groupby(['state_code','state','order_year']).sum()['total_sales'])
sd_totalstate=pd.DataFrame(sd_totalstate).sort_values('order_year')
sd_totalstate.reset_index(inplace=True)
sd_totalstate

,state_code,state,order_year,total_sales
0,AL,Alabama,2018,117812.8
1,ID,Idaho,2018,62752.2
2,OR,Oregon,2018,258532.9
3,OK,Oklahoma,2018,217234.1
4,IN,Indiana,2018,794171.1
...,...,...,...,...
130,MI,Mississippi,2020,36293.9
131,GA,Hawaii,2020,29292.4
132,OR,Oregon,2020,461918.1
133,IN,Kansas,2020,389544.7


In [6]:
#app dashboard using dash components drowpdown

app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="Select_year",
                 options=[
                     {"label": "2018", "value": 2018},
                     {"label": "2019", "value": 2019},
                     {"label": "2020", "value": 2020}],
                 multi=False,
                 value=2018,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='state_total_map', figure={})
])

# if __name__ == '__main__':
#     app.run_server(debug=True, use_reloader=False)

In [7]:
# app._terminate_server_for_port("localhost", 8050)

In [8]:
#Connecting the Dash Components & Graphs using call back 
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='state_total_map', component_property='figure')],
    [Input(component_id='Select_year', component_property='value')])

def update_graph(option_selected):
    print(option_selected)
    print(type(option_selected))

    container = "The year chosen by user was: {}".format(option_selected)
    sd_state=sd_totalstate.copy()
    sd_state=sd_state[sd_state['order_year'] == option_selected]
    #sd_state=sd_state[sd_state["state_code"] == "total_sales"]
    # sd_state=pd.DataFrame(sd_state)
    #sd_state['state']=sd_state['state'].astype(str)
    
    
    #choropleth map using plotly express 
    fig = px.choropleth(
        data_frame=sd_state,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='total_sales',
        hover_data=['state', 'total_sales'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'total_sales': 'Total sales by State'},
        template='plotly_dark')
    return container,fig
    # fig.show()
    
    # fig = go.Figure(data = go.Choropleth(
    #     locations = sd_state['state_code'],
    #     locationmode = 'USA-states',
    #     colorscale = 'Greens',
    #     text = sd_state['state_code'],
    #     z = sd-state['total_sales']),
    #     colorbar_title = "Total Sales by State"
    #                )
    # fig.update_layout(
    #     title_text = 'Total Sales by State',
    #     geo_scope = 'usa')

In [ ]:
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
2018
<class 'int'>


In [ ]:
# app._terminate_server_for_port("localhost", 8050)